---

<h1 style="text-align: center;font-size: 40px;">Prediction of Health Insurance Owners, who will be interested in Vehicle Insurance</h1>

---

<center><img src="https://www.cityinsurance.com.bd/wp-content/uploads/2020/02/How-to-Motor-Insurance-1-820x450.png "width="500" height="600"></center>

---

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import missingno as msno
import pandas_profiling
plt.style.use("fivethirtyeight")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/train.csv")
df.head()

In [ ]:
df.info()

> Checking is there any Null values or Not

In [ ]:
n = msno.bar(df,color="gold")

><h3>EDA</h3>

In [ ]:
sns.countplot("Gender",data=df)
plt.show()

>Observation:
- Dataset contains more number of Male than Female    

>Relation Between Gender and Response

In [ ]:
plt.style.use("fivethirtyeight")
sns.countplot(x = "Gender",data = df,hue = "Response")
plt.show()

>Observation:
- Dataset contains more number of Male than Female.So from that perspective the Response according to Gender(Male & Female) are Equal

>Response According to Age

In [ ]:
facet = sns.FacetGrid(df,hue="Response",aspect = 4)
facet.map(sns.kdeplot,"Age",shade = True)
facet.set(xlim = (0,df["Age"].max()))
facet.add_legend()
plt.show()

>Observation:
- Here we can see that, people ages between 39 to 50 are most likely to Response
- People ages between 20 to 30 are most likely to Not Response

>Relation of Driving license with Response

In [ ]:
sns.countplot("Driving_License",data = df,hue = "Response")
plt.show()

In [ ]:
df["Response"].value_counts()

>Observation:
- Number of People not having Driving License is very Small
- The number of Response is very small compared to Not Resonse

In [ ]:
sns.countplot(x = "Previously_Insured",data = df,hue = "Response")
plt.show()

In [ ]:
df.groupby("Response")["Previously_Insured"].value_counts()

>Observation:
- The people who were previously insured,i think they loose their interest of insured again,so there is a small number of people to Response
- The people who were not previously Insured ,there is certain number of people who are going to response

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
sns.countplot("Vehicle_Age", data = df,hue = "Response")
plt.xlabel("Fig: i",color="coral")
plt.subplot(1,2,2)
plt.rcParams['figure.figsize']=(6,8)
color = ['yellowgreen','gold',"lightskyblue"]
df['Vehicle_Age'].value_counts().plot.pie(y="Vehicle_Age",colors=color,explode=(0.02,0,0.3),startangle=50,shadow=True,autopct="%0.1f%%")
plt.axis('on')
plt.xlabel("Fig: ii",color="coral")
plt.show()

>Observation (Fig: i):
- People having Cars with age between 1-2 years are most likely to Response
- People having Cars with age greater than 2 years are less likely to response

>Observation (Fig: ii):
- 52.6% of the dataset contains cars with ages between 1-2 years
- 43.2% of the dataset contains cars with ages less than 1 years
- 4.2% of the dataset contains cars with ages greater than 2 years

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot("Vehicle_Damage", data = df,hue = "Response")
plt.show()

>Observation:
- People having their Vehicle damaged in the past are most likely to Response ("Experience speaks the Most" :They know the Consequences)
- People not having their Vehicle damaged in the past are most likely to not Response

In [ ]:
facet = sns.FacetGrid(df,hue="Response",aspect = 4)
facet.map(sns.kdeplot,"Annual_Premium",shade = True)
facet.set(xlim = (0,df["Annual_Premium"].max()))
facet.add_legend()
plt.show()

>Observation:
- According to Annual Premium the number of Response and not Response is the same
- Insurense with Annual premium less than 800,000 is most likely to respond

In [ ]:
facet = sns.FacetGrid(df,hue="Response",aspect = 4)
facet.map(sns.kdeplot,"Vintage",shade = True)
facet.set(xlim = (0,df["Vintage"].max()))
facet.add_legend()
plt.show()

In [ ]:
sns.catplot(x="Vehicle_Age", hue="Vehicle_Damage", col="Response",
                data=df, kind="count",
                height=6, aspect=.7)
plt.show()

>Observation:
- People Having vehicle Damage, vehicle with any age are most likely to Respond, so we can say that Vehicle Damage is the primary factor to Respond

In [ ]:
sns.catplot(x="Vehicle_Damage", hue="Previously_Insured", col="Response",
                data=df, kind="count",
                height=6, aspect=.7)
plt.show()

>Observation:
- People who were not previously insured are most likely to response after having Vehicle Damage
- People who were previously insured and had no vehicle damage currently , are most likely to not Response

><h3>Feature Engineering</h3>

In [ ]:
df.drop(["id","Region_Code","Policy_Sales_Channel"],axis = 1,inplace=True)
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["Vehicle_Damage"] = le.fit_transform(df["Vehicle_Damage"])

In [ ]:
df = pd.get_dummies(df,drop_first=True)
df.head()

In [ ]:
x = df.drop("Response",axis=1)
y = df["Response"]

>Finding the most important Features

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(x,y)

In [ ]:
plt.figure(figsize=(8,6))
important_features = pd.Series(model.feature_importances_,index = x.columns)
important_features.nlargest(7).plot(kind = "bar")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

><h3>Model Building</h3>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


>Decision Tree

In [ ]:
dt_clf = DecisionTreeClassifier(criterion='entropy',max_depth = 100,random_state=0)
dt_clf.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
dt_pred = dt_clf.predict(x_test)
dt_accuracy = accuracy_score(y_test,dt_pred)
dt_accuracy

>Random Forest

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 200,random_state=0)
rf_clf.fit(x_train,y_train)

In [ ]:
rf_pred = rf_clf.predict(x_test)
rf_accuracy = accuracy_score(y_test,rf_pred)
rf_accuracy

>Logistic Regression

In [ ]:
lr_clf = LogisticRegression(random_state=0)
lr_clf.fit(x_train,y_train)

In [ ]:
lr_pred = lr_clf.predict(x_test)
lr_accuracy = accuracy_score(y_test,lr_pred)
lr_accuracy

>LightGBM

In [ ]:
lgbm_clf = LGBMClassifier(n_estimators=1000,learning_rate=0.007,random_state=0)#1000
lgbm_clf.fit(x_train,y_train)

In [ ]:
lgbm_pred = lgbm_clf.predict(x_test)
lgbm_accuracy = accuracy_score(y_test,lgbm_pred)
lgbm_accuracy

>KNN

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=20)
knn_clf.fit(x_train,y_train)

In [ ]:
knn_pred = knn_clf.predict(x_test)
knn_accuracy = accuracy_score(y_test,knn_pred)
knn_accuracy

> <h3>All Models Accuracy</h3>

In [ ]:
acc_df = pd.DataFrame({"Decision Tree":dt_accuracy,"Random Forest":rf_accuracy,
                       "LightGBM":lgbm_accuracy,"Logistic Regression" : lr_accuracy,"KNN":knn_accuracy},index = ["Accuracy"])
acc_df.style.background_gradient(cmap = "Reds")

>Final Observation:
- Here i've used 5 ML algorithms(Decision Tree, Random Forest,LightGBM,Logistic Regression,KNN), among them LightGBM gives the highest accuracy, which is 87.7%

---

<h1 style="text-align: center;font-size: 20px;color: blue">Thanks for Reading!!</h1>

---